# Amazon Bedrock Knowledge Bases가 지원하는 쿼리 재구성

RAG 기반 GenAI 애플리케이션을 개발할 때 품질, 비용, 지연 시간을 최적화하는 것이 가장 중요한 요소 중 일부입니다. Foundation Model(FM)에 대한 입력 쿼리는 많은 질문과 복잡한 관계를 포함하는 매우 복잡한 경우가 많습니다. 이러한 복잡한 쿼리의 경우, 임베딩 단계에서 쿼리의 중요한 구성 요소가 가려지거나 희석될 수 있어, 쿼리의 모든 측면에 대한 컨텍스트를 제공하지 못하는 청크가 검색될 수 있습니다. 이로 인해 RAG 애플리케이션에서 원하는 것보다 낮은 품질의 응답이 생성될 수 있습니다.

이제 쿼리 재구성을 통해 복잡한 입력 프롬프트를 여러 개의 하위 쿼리로 분해할 수 있습니다. 이러한 하위 쿼리들은 각각 관련 청크를 검색하는 자체 검색 단계를 거치게 됩니다. 그 결과로 나온 청크들은 FM에 전달되어 응답을 생성하기 전에 함께 풀링되고 순위가 매겨집니다. 쿼리 재구성은 애플리케이션이 실제 운영 환경에서 직면할 수 있는 복잡한 쿼리의 정확도를 높이는 데 도움이 될 수 있는 또 다른 도구입니다.

# 1. Notebook setup
시작하기 위해 호환되는 역할과 컴퓨팅 환경으로 아래 단계들을 따르세요

In [1]:
%pip install --upgrade pip --quiet
%pip install -r ../requirements.txt --no-deps --quiet
%pip install -r ../requirements.txt --upgrade --quiet

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [2]:
# restart kernel
from IPython.core.display import HTML
HTML("<script>Jupyter.notebook.kernel.restart()</script>")

In [3]:
import os
import sys
import time
import boto3
import logging
import pprint
import json

# Set the path to import module
from pathlib import Path
current_path = Path().resolve()
current_path = current_path.parent
if str(current_path) not in sys.path:
    sys.path.append(str(current_path))
# Print sys.path to verify
# print(sys.path)

from utils.knowledge_base import BedrockKnowledgeBase

In [4]:
#Clients
s3_client = boto3.client('s3')
sts_client = boto3.client('sts')
session = boto3.session.Session()
region =  session.region_name
account_id = sts_client.get_caller_identity()["Account"]
bedrock_agent_client = boto3.client('bedrock-agent')
bedrock_agent_runtime_client = boto3.client('bedrock-agent-runtime') 
logging.basicConfig(format='[%(asctime)s] p%(process)s {%(filename)s:%(lineno)d} %(levelname)s - %(message)s', level=logging.INFO)
logger = logging.getLogger(__name__)
region, account_id

('us-west-2', '461433424192')

In [5]:
import time

# Get the current timestamp
current_time = time.time()

# Format the timestamp as a string
timestamp_str = time.strftime("%Y%m%d%H%M%S", time.localtime(current_time))[-7:]
# Create the suffix using the timestamp
suffix = f"{timestamp_str}"
knowledge_base_name_standard = 'standard-kb'
knowledge_base_description = "Octank 10k KB"
bucket_name = f'{knowledge_base_name_standard}-{suffix}'

foundation_model = "anthropic.claude-3-sonnet-20240229-v1:0"

data_source=[{"type": "S3", "bucket_name": bucket_name}]

## 2. 고정 청킹 전략으로 지식 베이스 생성

In [6]:
knowledge_base_standard = BedrockKnowledgeBase(
    kb_name=f'{knowledge_base_name_standard}-{suffix}',
    kb_description=knowledge_base_description,
    data_sources=data_source,
    chunking_strategy = "FIXED_SIZE", 
    suffix = f'{suffix}-f'
)

Step 1 - Creating or retrieving S3 bucket(s) for Knowledge Base documents
['standard-kb-1084148']
buckets_to_check:  ['standard-kb-1084148']
Creating bucket standard-kb-1084148
Step 2 - Creating Knowledge Base Execution Role (AmazonBedrockExecutionRoleForKnowledgeBase_1084148-f) and Policies
Step 3a - Creating OSS encryption, network and data access policies
Step 3b - Creating OSS Collection (this step takes a couple of minutes to complete)
{ 'ResponseMetadata': { 'HTTPHeaders': { 'connection': 'keep-alive',
                                         'content-length': '320',
                                         'content-type': 'application/x-amz-json-1.0',
                                         'date': 'Thu, 31 Jul 2025 08:41:51 '
                                                 'GMT',
                                         'x-amzn-requestid': '7db2b5c6-8a44-4ce1-9519-03183cacbcce'},
                        'HTTPStatusCode': 200,
                        'RequestId': '7db2b5c6-8a4

[2025-07-31 08:43:22,183] p2107 {base.py:258} INFO - PUT https://byl05aks5fxuf0gsen73.us-west-2.aoss.amazonaws.com:443/bedrock-sample-rag-index-1084148-f [status:200 request:0.355s]



Creating index:
{ 'acknowledged': True,
  'index': 'bedrock-sample-rag-index-1084148-f',
  'shards_acknowledged': True}
Step 4 - Will create Lambda Function if chunking strategy selected as CUSTOM
Not creating lambda function as chunking strategy is FIXED_SIZE
Step 5 - Creating Knowledge Base
{ 'createdAt': datetime.datetime(2025, 7, 31, 8, 44, 22, 307006, tzinfo=tzlocal()),
  'description': 'Octank 10k KB',
  'knowledgeBaseArn': 'arn:aws:bedrock:us-west-2:461433424192:knowledge-base/RQKFHHRKLO',
  'knowledgeBaseConfiguration': { 'type': 'VECTOR',
                                  'vectorKnowledgeBaseConfiguration': { 'embeddingModelArn': 'arn:aws:bedrock:us-west-2::foundation-model/amazon.titan-embed-text-v2:0'}},
  'knowledgeBaseId': 'RQKFHHRKLO',
  'name': 'standard-kb-1084148',
  'roleArn': 'arn:aws:iam::461433424192:role/AmazonBedrockExecutionRoleForKnowledgeBase_1084148-f',
  'status': 'CREATING',
  'storageConfiguration': { 'opensearchServerlessConfiguration': { 'collectionArn'

## 2.1 데이터셋을 Amazon S3에 업로드
이제 지식 베이스를 생성했으니, `Octank financial 10K` 보고서 데이터셋으로 채워보겠습니다. Knowledge Base 데이터 소스는 연결된 S3 버킷에서 데이터를 사용할 수 있어야 하며, `StartIngestionJob API` 호출을 사용하여 데이터의 변경 사항을 지식 베이스와 동기화할 수 있습니다. 이 예제에서는 우리의 헬퍼 클래스를 통해 API의 [boto3 추상화](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/bedrock-agent/client/start_ingestion_job.html)를 사용할 것입니다.

먼저 `dataset` 폴더에 있는 메뉴 데이터를 s3에 업로드해보겠습니다.

In [7]:
import os

def upload_directory(path, bucket_name):
    for root, dirs, files in os.walk(path):
        for file in files:
            file_to_upload = os.path.join(root, file)
            if file not in ["LICENSE", "NOTICE", "README.md"]:
                print(f"uploading file {file_to_upload} to {bucket_name}")
                s3_client.upload_file(file_to_upload, bucket_name, file)
            else:
                print(f"Skipping file {file_to_upload}")

upload_directory("../synthetic_dataset", bucket_name)


uploading file ../synthetic_dataset/2024_population.pdf to standard-kb-1084148


In [8]:
# ensure that the kb is available
time.sleep(30)
# sync knowledge base
knowledge_base_standard.start_ingestion_job()

job 1 started successfully

{ 'dataSourceId': '19GWGTVSR6',
  'ingestionJobId': 'J3RAHMTQUD',
  'knowledgeBaseId': 'RQKFHHRKLO',
  'startedAt': datetime.datetime(2025, 7, 31, 8, 44, 55, 472622, tzinfo=tzlocal()),
  'statistics': { 'numberOfDocumentsDeleted': 0,
                  'numberOfDocumentsFailed': 0,
                  'numberOfDocumentsScanned': 1,
                  'numberOfMetadataDocumentsModified': 0,
                  'numberOfMetadataDocumentsScanned': 0,
                  'numberOfModifiedDocumentsIndexed': 0,
                  'numberOfNewDocumentsIndexed': 1},
  'status': 'COMPLETE',
  'updatedAt': datetime.datetime(2025, 7, 31, 8, 45, 10, 262995, tzinfo=tzlocal())}
........................................

In [9]:
kb_id = knowledge_base_standard.get_knowledge_base_id()

'RQKFHHRKLO'


# 3. 쿼리 재구성 실제 적용
이 노트북에서는 쿼리 재구성의 이점을 얻을 수 있는 간단한 쿼리와 더 복잡한 쿼리를 조사하고, 이것이 생성된 응답에 어떤 영향을 미치는지 살펴보겠습니다.

## 복잡한 프롬프트

기능을 시연하기 위해, Octank 10K 재무 문서에 포함된 정보에 대해 여러 가지를 요청하는 쿼리를 살펴보겠습니다. 이 쿼리는 의미론적으로 관련되지 않은 몇 가지 요청을 포함하고 있습니다. 검색 단계에서 이 쿼리가 임베딩될 때, 쿼리의 일부 측면이 희석될 수 있으며, 따라서 반환되는 관련 청크가 이 복잡한 쿼리의 모든 구성 요소를 다루지 못할 수 있습니다.

지식 베이스를 쿼리하고 응답을 생성하기 위해 retrieve_and_generate API 호출을 사용할 것입니다. 쿼리 재구성 기능을 사용하기 위해, 아래와 같이 지식 베이스 구성에 추가 정보를 포함할 것입니다:

```
'orchestrationConfiguration': {
        'queryTransformationConfiguration': {
            'type': 'QUERY_DECOMPOSITION'
        }
    }
```
    
__참고__: 출력 응답 구조는 쿼리 재구성이 없는 일반 __retrieve_and_generate__와 동일합니다.

#### Without Query Reformulation

쿼리 재구성을 사용하지 않고 다음 쿼리에 대한 생성된 결과가 어떻게 나타나는지 살펴보겠습니다:

In [10]:
query = "2024년 시도별 인구・가구・주택 특성을 종합적으로 비교 분석하시오.(단, 각 시도의 연령구조, 1인가구 비율, 주택 노후도, 외국인 비율을 모두 포함하여 상위 3개 지역과 하위 3개 지역을 각각 제시하고, 이들 간의 특성 차이를 설명하시오)"

In [11]:
response_ret = bedrock_agent_runtime_client.retrieve_and_generate(
    input={
        "text": query
    },
    retrieveAndGenerateConfiguration={
        "type": "KNOWLEDGE_BASE",
        "knowledgeBaseConfiguration": {
            'knowledgeBaseId': kb_id,
            "modelArn": "arn:aws:bedrock:{}::foundation-model/{}".format(region, foundation_model),
            "retrievalConfiguration": {
                "vectorSearchConfiguration": {
                    "numberOfResults":5
                } 
            }
        }
    }
)


# generated text output

print(response_ret['output']['text'],end='\n'*2)

2024년 시도별 인구・가구・주택 특성을 종합적으로 비교해 보면, 상위 3개 지역은 서울, 경기, 인천이며 하위 3개 지역은 전남, 전북, 강원으로 나타났습니다.

상위 지역인 서울, 경기, 인천은 다음과 같은 특성을 보입니다:
- 1인 가구 비율이 높음 (서울 39.9%, 경기 32.5%, 인천 32.5%) (출처: 4)
- 고령 인구 비율이 낮음 (서울 15.7%, 경기 15.6%, 인천 15.3%) (출처: 3)
- 외국인 가구 비율이 높음 (출처: 1) 반면 하위 지역인 전남, 전북, 강원은 다음과 같은 특성을 보입니다:
- 1인 가구 비율이 낮음 (전남 25.8%, 전북 26.9%, 강원 27.6%) (출처: 4) 
- 고령 인구 비율이 높음 (전남 25.6%, 전북 23.8%, 강원 23.3%) (출처: 3)
- 주택 노후도가 높음 (자료 부족으로 구체적인 수치는 제시할 수 없음)

이처럼 상위 지역과 하위 지역 간에는 1인 가구 비율, 고령 인구 비율, 외국인 가구 비율 등에서 큰 차이를 보이고 있습니다. 상위 지역은 1인 가구와 외국인 가구 비율이 높고 고령 인구 비율이 낮은 반면, 하위 지역은 그 반대의 특성을 보이고 있습니다.



In [12]:
response_without_qr = response_ret['citations'][0]['retrievedReferences']
print("# of citations or chunks used to generate the response: ", len(response_without_qr))
def citations_rag_print(response_ret):
#structure 'retrievalResults': list of contents. Each list has content, location, score, metadata
    for num,chunk in enumerate(response_ret,1):
        print(f'Chunk {num}: ',chunk['content']['text'],end='\n'*2)
        print(f'Chunk {num} Location: ',chunk['location'],end='\n'*2)
        print(f'Chunk {num} Metadata: ',chunk['metadata'],end='\n'*2)

citations_rag_print(response_without_qr)

# of citations or chunks used to generate the response:  3
Chunk 1:  증가,     서울, 부산 등 11개 시도는 인구 감소      - (시군구) 전년 대비 87개 시군구 인구 증가,     142개 시군구 인구 감소     [권역별 인구]     - 1 -[가구]     2024년 11월 1일 기준 총가구는 2,300만 가구, 전년 대비 1.2% (27만 가구) 증가      ○ 일반가구는 총가구의 96.9%(2,229만 가구)로 전년 대비 1.0%(22만 가구) 증가      ○ 외국인가구 및 집단가구는 3.1%(70만 가구)로 전년 대비 7.3%(5만 가구) 증가      ※ 총가구 = 일반가구 + 외국인가구 + 집단가구     [총가구 및 연평균 증감률]          일반가구 중 1인 ․ 2인 가구는

Chunk 1 Location:  {'s3Location': {'uri': 's3://standard-kb-1084148/2024_population.pdf'}, 'type': 'S3'}

Chunk 1 Metadata:  {'x-amz-bedrock-kb-source-uri': 's3://standard-kb-1084148/2024_population.pdf', 'x-amz-bedrock-kb-document-page-number': 10.0, 'x-amz-bedrock-kb-chunk-id': '1%3A0%3ADIWoX5gBl5d7-3QYD9_x', 'x-amz-bedrock-kb-data-source-id': '19GWGTVSR6'}

Chunk 2:  51.4 0.5 16.3 15.7 -0.5 39.5 41.2 1.7 242.8 261.7 18.9     경 북 50.0 50.7 0.7 15.2 14.6 -0.5 35.9 37.9 2.0 236.8 259.2 22.4     경 남 47.8 48.5 0.7 16.6 16.0 -0.6 29.2 31.0 1.8 175.7 194.0 18.3  

위의 인용에서 볼 수 있듯이, 복잡한 쿼리를 사용한 우리의 검색은 건물과 관련된 청크는 전혀 반환하지 않고, 대신 내부고발자 사건과 가장 유사한 임베딩에만 초점을 맞추었습니다.

이는 쿼리의 임베딩 과정에서 쿼리의 해당 부분의 의미가 일부 희석되었을 수 있다는 것을 나타냅니다.

#### With Query Reformulation

이제 쿼리 재구성이 어떻게 더 정렬된 컨텍스트 검색에 도움이 될 수 있는지, 그리고 이것이 결과적으로 응답 생성의 정확성을 어떻게 향상시킬 수 있는지 살펴보겠습니다.

In [13]:
response_ret = bedrock_agent_runtime_client.retrieve_and_generate(
    input={
        "text": query
    },
    retrieveAndGenerateConfiguration={
        "type": "KNOWLEDGE_BASE",
        "knowledgeBaseConfiguration": {
            'knowledgeBaseId': kb_id,
            "modelArn": "arn:aws:bedrock:{}::foundation-model/{}".format(region, foundation_model),
            "retrievalConfiguration": {
                "vectorSearchConfiguration": {
                    "numberOfResults":5
                } 
            },
            'orchestrationConfiguration': {
                'queryTransformationConfiguration': {
                    'type': 'QUERY_DECOMPOSITION'
                }
            }
        }
    }
)


# generated text output

print(response_ret['output']['text'],end='\n'*2)

2024년 시도별 인구·가구·주택 특성을 종합적으로 비교해 보면, 상위 3개 지역은 서울, 경기, 인천이며 하위 3개 지역은 전남, 경북, 제주로 나타납니다.

상위 지역들은 다음과 같은 특성을 보입니다:
- 연령구조에서 생산연령인구(15~64세) 비중이 높음
- 1인가구 비율이 높음 (서울 39.9%, 경기 36.6%, 인천 32.5%)
- 주택 노후도(30년 이상 주택 비율)가 낮음
- 외국인 비율이 높음 (경기 13.5%, 서울 10.8%, 인천 9.9%) 반면 하위 지역들은 다음과 같은 특성이 있습니다:
- 고령인구(65세 이상) 비중이 높음 (전남 25.1%, 경북 26.5%, 제주 26.5%)  
- 1인가구 비율이 낮음 (전남 28.0%, 경북 28.0%, 제주 25.7%)
- 주택 노후도가 높음 (전남 42.8%, 경북 37.7%, 제주 자료 없음)
- 외국인 비율이 낮음 (전남 1.7%, 경북 2.4%, 제주 0.0%)



쿼리 재구성을 사용하여 검색된 청크들을 살펴보겠습니다

In [14]:
response_with_qr = response_ret['citations'][0]['retrievedReferences']
print("# of citations or chunks used to generate the response: ", len(response_with_qr))
def citations_rag_print(response_ret):
#structure 'retrievalResults': list of contents. Each list has content, location, score, metadata
    for num,chunk in enumerate(response_ret,1):
        print(f'Chunk {num}: ',chunk['content']['text'],end='\n'*2)
        print(f'Chunk {num} Location: ',chunk['location'],end='\n'*2)
        print(f'Chunk {num} Metadata: ',chunk['metadata'],end='\n'*2)

citations_rag_print(response_with_qr)
print("# of citations or chunks used to generate the response: ", len(response_with_qr))

# of citations or chunks used to generate the response:  4
Chunk 1:  51.4 0.5 16.3 15.7 -0.5 39.5 41.2 1.7 242.8 261.7 18.9     경 북 50.0 50.7 0.7 15.2 14.6 -0.5 35.9 37.9 2.0 236.8 259.2 22.4     경 남 47.8 48.5 0.7 16.6 16.0 -0.6 29.2 31.0 1.8 175.7 194.0 18.3     제 주 45.1 45.9 0.7 18.6 18.0 -0.7 25.1 26.5 1.4 134.9 147.7 12.8     - 20 -< 표 9 > 시도별 인구의 연령 분포, 2023~2024년     (단위 : 천 명, %, %p)     시 도 0~14세(유소년) 15~64세(생산연령) 65세 이상(고령)     2023년 2024년

Chunk 1 Location:  {'s3Location': {'uri': 's3://standard-kb-1084148/2024_population.pdf'}, 'type': 'S3'}

Chunk 1 Metadata:  {'x-amz-bedrock-kb-source-uri': 's3://standard-kb-1084148/2024_population.pdf', 'x-amz-bedrock-kb-document-page-number': 29.0, 'x-amz-bedrock-kb-chunk-id': '1%3A0%3AUIWoX5gBl5d7-3QYD9_y', 'x-amz-bedrock-kb-data-source-id': '19GWGTVSR6'}

Chunk 2:  시도별 1인가구 비율, 2019~2024년 (단위 : %, %p)     시 도 2019년 2020년 2021년 2022년 2023년 2024년 2023년     대비 증감     전 국 30.2 31.7 33.4 34.5 35.5 36.1 0.6     서 울 33.4 34.9 36.8 38.2 39.3 3

쿼리 재구성을 활성화했을 때, 검색된 청크들이 이제 내부고발자 스캔들과 워터프론트 건물의 위치 구성 요소 모두에 대한 컨텍스트를 제공하는 것을 볼 수 있습니다.

### CloudWatch Logs를 사용하여 프롬프트 분해 관찰하기
복잡한 쿼리는 검색을 수행하기 전에 여러 개의 하위 쿼리로 분해됩니다. 위 예제 쿼리에서 분해 작업을 위한 호출을 분리했을 때 이를 확인할 수 있으며, 여기서 __standalone_question__은 원래 쿼리이고 결과로 나온 하위 쿼리들은 __\<query\>__ 태그 사이에 표시됩니다.

__참고__: CloudWatch에서 로그를 볼 수 있으려면 Bedrock에서 호출 로깅을 활성화해야 합니다. 자세한 내용은 [여기](https://docs.aws.amazon.com/bedrock/latest/userguide/model-invocation-logging.html)를 참조하세요.

```
<generated_queries>

<standalone_question>
What is octank tower and how does the whistleblower scandal hurt the company and its image?
</standalone_question>

<query>
What is octank tower?
</query>

<query>
What is the whistleblower scandal involving Octank company?
</query>

<query>
How did the whistleblower scandal affect Octank company's reputation and public image?
</query>

</generated_queries>
```

```
<generated_queries>

<standalone_question>
옥탱크 타워는 무엇이고 내부고발자 스캔들이 회사와 그 이미지에 어떤 해를 끼쳤습니까?
</standalone_question>

<query>
옥탱크 타워는 무엇입니까?
</query>

<query>
옥탱크 회사와 관련된 내부고발자 스캔들은 무엇입니까?
</query>

<query>
내부고발자 스캔들이 옥탱크 회사의 평판과 공공 이미지에 어떤 영향을 미쳤습니까?
</query>

</generated_queries>
```

<div class="alert alert-block alert-warning">
<b>참고:</b> 불필요한 요금이 발생하지 않도록 KB(Knowledge Base), OSS 인덱스 및 관련 IAM 역할과 정책을 삭제하는 것을 잊지 마세요.
</div>

In [15]:
print("===============================Knowledge base with fixed chunking==============================\n")
knowledge_base_standard.delete_kb(delete_s3_bucket=True, delete_iam_roles_and_policies=True)

===============================Knowledge base with fixed chunking==============================

Deleted data source 19GWGTVSR6
======== Knowledge base and all data sources deleted =========
Found bucket standard-kb-1084148
Deleted all objects in bucket standard-kb-1084148
Deleted bucket standard-kb-1084148
======== S3 bucket deletion process completed =========
Found role AmazonBedrockExecutionRoleForKnowledgeBase_1084148-f
======Attached policies with role AmazonBedrockExecutionRoleForKnowledgeBase_1084148-f========
 [{'PolicyName': 'AmazonBedrockOSSPolicyForKnowledgeBase_1084148-f', 'PolicyArn': 'arn:aws:iam::461433424192:policy/AmazonBedrockOSSPolicyForKnowledgeBase_1084148-f'}, {'PolicyName': 'AmazonBedrockS3PolicyForKnowledgeBase_1084148-f', 'PolicyArn': 'arn:aws:iam::461433424192:policy/AmazonBedrockS3PolicyForKnowledgeBase_1084148-f'}, {'PolicyName': 'AmazonBedrockCloudWatchPolicyForKnowledgeBase_1084148-f', 'PolicyArn': 'arn:aws:iam::461433424192:policy/AmazonBedrockCloudWatch

이제 쿼리 재구성이 어떻게 작동하는지, 그리고 복잡한 쿼리에 대한 응답을 어떻게 개선할 수 있는지 살펴보았으니, 이 기술을 더 깊이 탐구하고 실험하여 RAG 워크플로우를 최적화해 보시기 바랍니다.